In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd ./drive/MyDrive/LELSD/notebooks/

/content/drive/MyDrive/LELSD/notebooks


In [16]:
# !pip install boto3 botocore awscli --ignore-installed
# !pip install ninja
# !pip install torch==1.9.0
# !pip install torchvision==0.10.0

In [4]:
import copy
import os

import click
import imageio
import numpy as np
import PIL.Image
import torch
import torch.nn.functional as F


import sys
sys.path.append("../")
import models
from lelsd import LELSD
from utils.stylegan1_utils import StyleGAN1SampleGenerator
from utils.stylegan2_utils import StyleGAN2SampleGenerator
from utils.stylegan3_utils import StyleGAN3SampleGenerator
from utils.segmentation_utils import FaceSegmentation
# from spaceUtils import * 

import cv2

# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, './stylegan2-ada-pytorch')

import dnnlib
import legacy
import matplotlib.pyplot as plt

plt.style.use('fivethirtyeight')

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [5]:
from tqdm import tqdm
def computeScore(m1,m2):
    n=np.sum(np.logical_and(np.array(m1),np.array(m2)))
    u=np.sum(np.logical_or(np.array(m1),np.array(m2)))
    return u/n
    
def load_pretrained_model(model_name, dataset_name):
    if model_name != "biggan":
        G = models.get_model(model_name,
                             f"../pretrained/{model_name}/{model2available_dataset[model_name][dataset_name]}")
    else:
        G = models.get_model(model_name, model2available_dataset[model_name][dataset_name])
    return G

In [6]:
model2available_dataset = {
    "stylegan2": {
        "FFHQ": "ffhq.pkl"
    },
    "stylegan3": {
        "FFHQ": "stylegan3-t-ffhq-1024x1024.pkl"
    }
}

dataset_name   = 'FFHQ'
truncation_psi = 0.7

In [7]:
device     = torch.device('cuda')
G2   = load_pretrained_model('stylegan2', dataset_name)
SG2 = StyleGAN2SampleGenerator(G=G2, device=device, truncation_psi=truncation_psi)

G3   = load_pretrained_model('stylegan3', dataset_name)
SG3 = StyleGAN3SampleGenerator(G=G3, device=device, truncation_psi=truncation_psi)

### Edit Quantifiying 1: Compute spatial change as a function of alpha

In [8]:
from utils.segmentation_utils import FaceSegmentation
face_bisenet      = models.get_model("face_bisenet", "../pretrained/face_bisenet/model.pth")
face_segmentation = FaceSegmentation(face_bisenet=face_bisenet, device=device)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [ ]:
ys2 = SG2.generate_batch_from_seeds([504], return_style=True)
ys3 = SG3.generate_batch_from_seeds([1002], return_style=True)

In [ ]:
ret2 = {}
alphas = [0,5,10,15,20,25,30,35,40]
img2 = editGan2(SG2 ,ys2['ys'], ys2['rgb_ys'], 0, 175, 4)
mask2 = (face_segmentation.predict(img2) == 17)[0,0]
for alpha in tqdm(alphas):
    modifimg2 = editGan2(SG2 ,ys2['ys'], ys2['rgb_ys'], -alpha, 175, 4)
    mask1 = (face_segmentation.predict(modifimg2) == 17)[0,0]
    

    ret2[alpha] = computeScore(mask1, mask2)

In [ ]:
ret3 = {}
alphas = [0,5,10,15,20,25,30,35,40]
img3 = editGan3(SG3 ,ys['ys'], 0, 212, 6)
mask3 = (face_segmentation.predict(img3) == 17)[0,0]
for alpha in tqdm(alphas):
    modifimg3 = editGan3(SG3 ,ys['ys'], alpha, 212, 6)
    mask1 = (face_segmentation.predict(modifimg3) == 17)[0,0]

    ret3[alpha] = computeScore(mask1, mask3)

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(alphas,list(ret2.values()), label='SG2')
plt.plot(alphas,list(ret3.values()), label='SG3')
plt.plot(alphas,list(ret3.values()), 'o')
plt.plot(alphas,list(ret2.values()), 'o')
plt.title('Union over intersection across models for a modification')
plt.xlabel('alpha values')
plt.xlabel('union over intersection score')
plt.legend()

In [ ]:
fig,ax = plt.subplots(1,4, figsize=(22,5))
ax[0].imshow(img2)
ax[1].imshow(modifimg2)
ax[2].imshow(img3)
ax[3].imshow(modifimg3)

ax[0].set_title('SG2 | 175_4 | alpha=0')
ax[1].set_title('SG2 | 175_4 | alpha=+40')
ax[2].set_title('SG3 | 212_6 | alpha=0')
ax[3].set_title('SG3 | 212_6 | alpha=+40')
ax[0].axis('off')
ax[1].axis('off')
ax[2].axis('off')
ax[3].axis('off')
plt.suptitle('sensitivity of SG2 versus SG3')

### Edit Quantifiying 2: Compute Discriminant score change as a function of alpha

In [15]:
from models.stylegan3 import dnnlib
from models.stylegan3 import legacy

model_path = "../pretrained/stylegan2/ffhq.pkl"
with dnnlib.util.open_url(model_path) as f:
    D2 = legacy.load_network_pkl(f)['D']
D2 = D2.eval()

# here we use SG3T
model_path = "../pretrained/stylegan3/stylegan3-t-ffhq-1024x1024.pkl"
with dnnlib.util.open_url(model_path) as f:
    D3 = legacy.load_network_pkl(f)['D']
D3 = D3.eval()

In [17]:
ys2 = SG2.generate_batch_from_seeds([504], return_style=True)
ys3 = SG3.generate_batch_from_seeds([1002], return_style=True)

Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.
Setting up PyTorch plugin "filtered_lrelu_plugin"... Done.


In [ ]:
plot the image original ones to see them

In [ ]:
dscoreS2 = []
alphas  = [0,5,10,15,20,25,30,35,40]

for alpha in tqdm(alphas):
    modifimg2 = editGan2(SG2 ,ys2['ys'], ys2['rgb_ys'], -alpha, 175, 4)

    dscoreS2.append(D2(modifimg2,None))
    

In [ ]:
dscoreS3 = []
alphas  = [0,5,10,15,20,25,30,35,40]

for alpha in tqdm(alphas):
    modifimg3 = editGan3(SG2 ,ys2['ys'], -alpha, 175, 4)

    dscoreS3.append(D3(modifimg2,None))
    

In [ ]:
plt.plot()
plt.imshow()